## Language Translation Using sequence to sequence 

In [1]:
#importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_s = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dimension = 256  # Latent dimensionality of the encoding space.
number_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
# data_path = '/content/french-eng/fra.txt'

In [ ]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
